In [5]:
# ----------------------------------------------------------------------
# 1. 필수 라이브러리 임포트
# ----------------------------------------------------------------------
import os
import re
import datetime
import random
import time
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

# PyTorch 및 Hugging Face
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from datasets import Dataset as HFDataset

# ✨ LoRA (PEFT)
from peft import get_peft_model, LoraConfig, TaskType, PeftModel

# ✨ 성능 지표
from sklearn.metrics import classification_report

# ----------------------------------------------------------------------
# 2. 유틸리티 함수 정의
# ----------------------------------------------------------------------

def format_time(elapsed):
    """
    경과 시간을 HH:MM:SS 형식으로 포맷
    """
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

# ----------------------------------------------------------------------
# 3. 데이터 로딩 및 전처리 (수정: 인덱스 컬럼 안전 제거 로직 추가)
# ----------------------------------------------------------------------

# 🌟 중요: 경로 설정
MODEL_LOAD_PATH = os.path.abspath('./model_checkpoint') 
LORA_SAVE_PATH = os.path.abspath('./peft_model_checkpoint')
TEST_DATA_PATH = './data/test.csv' 

try:
    # 저장된 모델 경로에서 토크나이저 로드 시도
    tokenizer = BertTokenizer.from_pretrained(MODEL_LOAD_PATH)
except Exception:
    # 로컬 로드 실패 시 원본 KR-BERT 경로로 폴백 (토크나이저 Vocab 일치를 위해)
    print("경고: 로컬 토크나이저 로드 실패, 원본 KR-BERT 경로 사용")
    tokenizer = BertTokenizer.from_pretrained("snunlp/KR-BERT-char16424")

MAX_LENGTH = 512

def load_and_preprocess_data(csv_path, is_training=True):
    """
    CSV 파일 로드 및 Hugging Face Dataset 객체로 변환
    """
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"'{csv_path}' 파일을 찾을 수 없습니다.")

    print(f"\n[데이터 로드] {csv_path}에서 데이터 로드 중...")
    
    try:
        df = pd.read_csv(csv_path, encoding='utf-8-sig')
    except:
        df = pd.read_csv(csv_path, encoding='cp949')
        
    # 🌟 컬럼명 매핑 (훈련/테스트 데이터에 따라 다름)
    if is_training:
        # 훈련 데이터 컬럼명 (stage1_data.csv 기준)
        text_col, label_col = 'content', 'label'
    else: 
        # 테스트 데이터 컬럼명 (image_70b35e.png 기준)
        text_col, label_col = '문장', '비도덕성 여부'
        
    required_cols = [text_col, label_col] + ([] if is_training else ['유형'])
    
    # 필수 컬럼 검사
    if not all(col in df.columns for col in required_cols):
        missing_cols = [col for col in required_cols if col not in df.columns]
        raise ValueError(f"CSV 파일에 다음 필수 컬럼이 없습니다: {missing_cols}")

    df = df[required_cols].dropna().reset_index(drop=True)
    
    # 컬럼 통일
    df.columns = ['text', 'label'] + ([] if is_training else ['type'])
    
    # 테스트 데이터 레이블 매핑 (FALSE=0, TRUE=1)
    if not is_training and df['label'].dtype == object:
        df['label'] = df['label'].astype(str).str.upper().map({'FALSE': 0, 'TRUE': 1}).fillna(-1)
    
    df['text'] = df['text'].astype(str)

    # 🌟🌟🌟 강력한 텍스트 정제 로직 🌟🌟🌟
    print("✅ 텍스트 데이터에서 유니코드 제어 문자 및 손상된 문자 정제 중...")
    control_char_regex = r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F-\x9F]'
    df['text'] = df['text'].str.replace(r'@[\w\d#]+', ' ', regex=True)
    df['text'] = df['text'].str.replace(control_char_regex, '', regex=True)
    df['text'] = df['text'].str.replace(r'[^\w\s\.\,\!\?]', ' ', regex=True) 
    df['text'] = df['text'].str.replace(r'\s+', ' ', regex=True).str.strip()
    df = df[df['text'].str.strip().astype(bool)].reset_index(drop=True)
    
    # 🌟🌟🌟 강력한 레이블 정제 로직 🌟🌟🌟
    valid_labels = {0, 1}
    numeric_labels = pd.to_numeric(df['label'], errors='coerce')
    
    df_cleaned = df[numeric_labels.notna() & (numeric_labels == numeric_labels.astype(int))]
    df_cleaned['label'] = df_cleaned['label'].astype(int)
    
    df_final = df_cleaned[df_cleaned['label'].isin(valid_labels)].reset_index(drop=True)
    invalid_count = len(df) - len(df_final)
    
    if invalid_count > 0:
        print(f"✅ 레이블 정제 완료: 유효하지 않은 값/레이블({valid_labels} 외) {invalid_count}개 행을 제거했습니다.")
        
    df = df_final
    # 🌟🌟🌟 레이블 정제 로직 끝 🌟🌟🌟

    hf_dataset = HFDataset.from_pandas(df)
    print(f"총 데이터 수: {len(hf_dataset)}")
    print(f"레이블 분포:\n{df['label'].value_counts()}")
    
    # 🌟🌟🌟 안전 로직: 인덱스 컬럼 제거 🌟🌟🌟
    if '__index_level_0__' in hf_dataset.column_names:
        hf_dataset = hf_dataset.remove_columns(['__index_level_0__'])
    # 🌟🌟🌟 안전 로직 끝 🌟🌟🌟
    
    # 테스트 데이터의 경우 'type' 컬럼을 데이터프레임으로 별도 반환하여 원본 맵핑 유지
    if not is_training:
        return hf_dataset, df['type'].tolist()
        
    return hf_dataset

def tokenize_function(examples):
    """
    토크나이징 함수
    """
    return tokenizer(
        examples["text"],
        max_length=MAX_LENGTH,
        truncation=True,
        padding="max_length"
    )

# ----------------------------------------------------------------------
# 4. LoRA 모델 및 수동 디버깅 루프 설정 (훈련)
# ----------------------------------------------------------------------

def debug_lora_training(epochs=3, batch_size=8, save_path=LORA_SAVE_PATH):
    
    # 1. GPU 및 환경 설정
    num_gpus = torch.cuda.device_count()
    if torch.cuda.is_available():
        DEVICE = torch.device("cuda:0") # cuda:0 장치 명시
    else:
        DEVICE = torch.device("cpu")

    print(f"\n=== 디버깅 환경 설정 ===")
    print(f"사용 가능 GPU 수: {num_gpus}")
    print(f"**최종 학습 장치**: {DEVICE}")
    print(f"배치 크기: {batch_size}")
    print("--------------------------------")

    # 2. 데이터 준비
    raw_dataset = load_and_preprocess_data(csv_path='stage1_data.csv', is_training=True) 
    split_dataset = raw_dataset.train_test_split(test_size=0.1, seed=42)
    tokenized_train_dataset = split_dataset['train'].map(tokenize_function, batched=True)
    
    # TensorDataset으로 변환
    input_ids = np.array(tokenized_train_dataset['input_ids']).astype(np.int64)
    attention_mask = np.array(tokenized_train_dataset['attention_mask']).astype(np.int64)
    token_type_ids = np.array(tokenized_train_dataset['token_type_ids']).astype(np.int64)
    labels = np.array(tokenized_train_dataset['label']).astype(np.int64) 
    
    train_data = TensorDataset(
        torch.tensor(input_ids).long(), 
        torch.tensor(attention_mask).long(), 
        torch.tensor(token_type_ids).long(), 
        torch.tensor(labels).long()
    )
    
    train_dataloader = DataLoader(
        train_data, 
        sampler=RandomSampler(train_data), 
        batch_size=batch_size
    )

    # 3. 모델 로드 및 LoRA 적용
    print(f"\n[모델 로드] 저장된 모델 '{MODEL_LOAD_PATH}' 로드 중...")
    
    try:
        # 1. Base Model을 단일 장치에 수동 로드
        base_model = BertForSequenceClassification.from_pretrained(
            MODEL_LOAD_PATH, 
            num_labels=2,
            torch_dtype=torch.float32, 
            return_dict=True           
        )
    
        # 2. LoRA 설정 및 적용
        lora_config = LoraConfig(r=8, lora_alpha=16, target_modules=["query", "key", "value"], 
                                 lora_dropout=0.05, bias="none", task_type=TaskType.SEQ_CLS)
        lora_model = get_peft_model(base_model, lora_config)
    
        # 3. 모델을 최종 학습 장치로 이동
        lora_model.to(DEVICE)
        print("✅ 모델 가중치 로드 및 단일 장치 이동 성공.")
    
    except Exception as e:
        print(f"❌ 치명적인 모델 로드 오류: {e}")
        return None

    print("--- LoRA 적용 모델 정보 ---")
    lora_model.print_trainable_parameters()
    
    # 4. 옵티마이저 설정 (수동)
    optimizer = torch.optim.AdamW(lora_model.parameters(), lr=2e-5, eps=1e-8)
    
    # 5. 수동 학습 루프 시작
    total_t0 = time.time()
    
    for epoch_i in range(0, epochs):
        print(f"\n======== Epoch {epoch_i + 1} / {epochs} ========")
        print('Training...')

        t0 = time.time()
        lora_model.train()
        
        for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
            
            try:
                # 텐서 GPU 이동
                b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch
                
                b_input_ids = b_input_ids.to(DEVICE)
                b_input_mask = b_input_mask.to(DEVICE)
                b_token_type_ids = b_token_type_ids.to(DEVICE)
                b_labels = b_labels.to(DEVICE) 
                
                lora_model.zero_grad()

                outputs = lora_model(
                    b_input_ids, 
                    token_type_ids=b_token_type_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels
                )
                
                loss = outputs.loss

                # 역전파 및 최적화
                loss.backward()
                torch.nn.utils.clip_grad_norm_(lora_model.parameters(), 1.0)
                optimizer.step()

            except Exception as e:
                # 🚨 에러 발생 시 디버깅 정보 출력 🚨
                print("\n\n" + "="*80)
                print(f"🚨🚨🚨 런타임/CUDA 에러 발생: {e} 🚨🚨🚨")
                print(f"🚨 에러 발생 지점: Epoch {epoch_i + 1}, Step {step}, Batch Size: {batch_size}")
                print(f"\n[디버깅 정보 - 레이블 확인]")
                print(f"배치 레이블(CPU): {b_labels.cpu().tolist()}")
                print(f"배치 레이블 최소/최대값: {b_labels.min().item()} / {b_labels.max().item()}")
                print("="*80 + "\n")
                
                return lora_model 

        print(f"Epoch {epoch_i + 1} Training Time: {format_time(time.time() - t0)}")
        
    print("\nTraining complete! (수동 루프)")
    
    # 6. 훈련된 LoRA 어댑터 저장
    print(f"\n=== 6. LoRA 어댑터 저장 ===")
    lora_model.save_pretrained(save_path)
    print(f"✅ LoRA 어댑터가 '{save_path}'에 저장되었습니다.")
    
    return lora_model

# ----------------------------------------------------------------------
# 5. 모델 추론 및 유형별 평가 함수 추가 (수정: 콤마 분리 로직 적용)
# ----------------------------------------------------------------------

def inference_and_evaluate(lora_save_path, test_csv_path, batch_size=32):
    
    if not os.path.exists(lora_save_path):
        print(f"경고: LoRA 어댑터 경로 '{lora_save_path}'를 찾을 수 없습니다. 평가를 건너뜁니다.")
        return

    # 1. GPU 및 환경 설정
    if torch.cuda.is_available():
        DEVICE = torch.device("cuda:0")
    else:
        DEVICE = torch.device("cpu")
    print(f"\n\n=== 🧪 추론 및 평가 시작 ===")
    print(f"**최종 추론 장치**: {DEVICE}")
    
    # 2. Base Model 로드 및 LoRA 어댑터 적용
    try:
        base_model = BertForSequenceClassification.from_pretrained(
            MODEL_LOAD_PATH, 
            num_labels=2,
            return_dict=True
        )
    except Exception as e:
        print(f"❌ Base Model 로드 실패: {e}")
        return
    
    lora_model = PeftModel.from_pretrained(base_model, lora_save_path)
    lora_model.eval() 
    lora_model.to(DEVICE)
    print("✅ LoRA 어댑터가 Base Model에 성공적으로 병합되었습니다.")
    
    # 3. 테스트 데이터 로드 및 토크나이징
    try:
        test_dataset_hf, original_types = load_and_preprocess_data(
            test_csv_path, is_training=False
        )
    except Exception as e:
        print(f"❌ 테스트 데이터 로드/전처리 실패: {e}")
        return
        
    tokenized_test_dataset = test_dataset_hf.map(tokenize_function, batched=True)
    
    # 4. DataLoader 준비
    test_input_ids = torch.tensor(tokenized_test_dataset['input_ids']).long()
    test_attention_mask = torch.tensor(tokenized_test_dataset['attention_mask']).long()
    test_token_type_ids = torch.tensor(tokenized_test_dataset['token_type_ids']).long()
    test_labels = torch.tensor(tokenized_test_dataset['label']).long()
    
    test_data = TensorDataset(test_input_ids, test_attention_mask, test_token_type_ids, test_labels)
    test_dataloader = DataLoader(
        test_data, 
        sampler=SequentialSampler(test_data), 
        batch_size=batch_size
    )
    
    # 5. 추론 루프 실행
    predictions = []
    true_labels = []
    
    print("\n[추론] 모델 예측 수행 중...")
    
    for batch in tqdm(test_dataloader, desc="Inference"):
        batch = tuple(t.to(DEVICE) for t in batch)
        b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch
        
        with torch.no_grad():
            outputs = lora_model(
                b_input_ids, 
                token_type_ids=b_token_type_ids, 
                attention_mask=b_input_mask
            )
            
        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        predictions.extend(np.argmax(logits, axis=1).flatten())
        true_labels.extend(label_ids.flatten())

    # 6. 유형별 결과 분석 (핵심 수정 부분)
    
    # LoRA 어댑터 적용이 성공했으므로, 여기서 테스트 데이터프레임을 다시 로드하여
    # '유형' 컬럼을 안전하게 분리합니다. (정제된 데이터의 맵핑을 사용)
    
    # 정제 후 남은 데이터프레임 구조를 복원
    results_df = pd.DataFrame({
        'type': original_types[:len(true_labels)], # 정제 후 남은 데이터만큼만 사용
        'true_label': true_labels,
        'prediction': predictions
    })
    
    # 🌟🌟🌟 유형 컬럼을 콤마(,)로 분리하고 모든 고유 유형을 추출 🌟🌟🌟
    all_types = set()
    results_df['type'].astype(str).str.split(',').apply(all_types.update)
    all_types.discard('nan')
    
    print("\n\n=== 📊 다중 유형별 분류 결과 (콤마 분리) ===")
    
    # 전체 보고서
    print("\n[전체 데이터셋 분류 보고서]")
    print(classification_report(results_df['true_label'], results_df['prediction'], 
                                target_names=['비도덕성 없음 (0)', '비도덕성 있음 (1)']))
    
    # 🌟 유형별 보고서 (각 유형에 해당되는 샘플만 평가)
    for type_name in sorted(list(all_types)):
        
        # 현재 type_name을 포함하는 행만 필터링
        group = results_df[results_df['type'].astype(str).str.contains(type_name)].reset_index(drop=True)
        
        if len(group) == 0:
            continue
            
        print(f"\n--- [유형별 테스트 결과] 유형: {type_name} ---")
        print(f"총 샘플 수: {len(group):,}")
        
        try:
            report = classification_report(group['true_label'], group['prediction'], 
                                           target_names=['비도덕성 없음 (0)', '비도덕성 있음 (1)'], 
                                           output_dict=False, zero_division=0)
            print(report)
        except ValueError as e:
            print(f"경고: {type_name}에서 평가 오류 발생: {e}")
            
    print("================================")
    print("✅ 다중 유형별 추론 및 평가가 완료되었습니다.")
    
    return


# ----------------------------------------------------------------------
# 6. 메인 실행부
# ----------------------------------------------------------------------
if __name__ == '__main__':
    try:
        # 1. 모델 훈련 및 LoRA 어댑터 저장
        final_model = debug_lora_training(epochs=3, batch_size=8)
        
        if final_model is not None:
            print("\n모든 훈련 및 저장 작업이 완료되었습니다.")
            
            # 2. 훈련된 모델로 테스트 데이터 추론 및 유형별 평가 수행
            inference_and_evaluate(
                lora_save_path=LORA_SAVE_PATH,
                test_csv_path=TEST_DATA_PATH,
                batch_size=32 
            )
            
    except Exception as e:
        print(f"\n최종 실행 실패: {e}")


=== 디버깅 환경 설정 ===
사용 가능 GPU 수: 2
**최종 학습 장치**: cuda:0
배치 크기: 8
--------------------------------

[데이터 로드] stage1_data.csv에서 데이터 로드 중...
✅ 텍스트 데이터에서 유니코드 제어 문자 및 손상된 문자 정제 중...
총 데이터 수: 199999
레이블 분포:
label
0    100000
1     99999
Name: count, dtype: int64


Map:   0%|          | 0/179999 [00:00<?, ? examples/s]


[모델 로드] 저장된 모델 '/home/kds/pytorch/CheckViolence/finalProject/model_checkpoint' 로드 중...
✅ 모델 가중치 로드 및 단일 장치 이동 성공.
--- LoRA 적용 모델 정보 ---
trainable params: 443,906 || all params: 99,100,420 || trainable%: 0.4479

======== Epoch 1 / 3 ========
Training...


Training: 100%|██████████| 22500/22500 [1:30:41<00:00,  4.13it/s]


Epoch 1 Training Time: 1:30:41

======== Epoch 2 / 3 ========
Training...


...

Epoch 3 Training Time: 1:31:08

Training complete! (수동 루프)

=== 6. LoRA 어댑터 저장 ===
✅ LoRA 어댑터가 '/home/kds/pytorch/CheckViolence/finalProject/peft_model_checkpoint'에 저장되었습니다.

모든 훈련 및 저장 작업이 완료되었습니다.


=== 🧪 추론 및 평가 시작 ===
**최종 추론 장치**: cuda:0
✅ LoRA 어댑터가 Base Model에 성공적으로 병합되었습니다.

[데이터 로드] ./data/test.csv에서 데이터 로드 중...
✅ 텍스트 데이터에서 유니코드 제어 문자 및 손상된 문자 정제 중...
총 데이터 수: 44998
레이블 분포:
label
1    26011
0    18987
Name: count, dtype: int64


Map:   0%|          | 0/44998 [00:00<?, ? examples/s]


[추론] 모델 예측 수행 중...


Inference: 100%|██████████| 1407/1407 [08:23<00:00,  2.80it/s]




=== 📊 다중 유형별 분류 결과 (콤마 분리) ===

[전체 데이터셋 분류 보고서]
              precision    recall  f1-score   support

 비도덕성 없음 (0)       0.73      0.53      0.61     18987
 비도덕성 있음 (1)       0.71      0.85      0.78     26011

    accuracy                           0.72     44998
   macro avg       0.72      0.69      0.69     44998
weighted avg       0.72      0.72      0.71     44998


--- [유형별 테스트 결과] 유형: ABUSE ---
총 샘플 수: 1,474
              precision    recall  f1-score   support

 비도덕성 없음 (0)       0.00      0.00      0.00         0
 비도덕성 있음 (1)       1.00      0.89      0.94      1474

    accuracy                           0.89      1474
   macro avg       0.50      0.44      0.47      1474
weighted avg       1.00      0.89      0.94      1474


--- [유형별 테스트 결과] 유형: CENSURE ---
총 샘플 수: 19,866
              precision    recall  f1-score   support

 비도덕성 없음 (0)       0.00      0.00      0.00         0
 비도덕성 있음 (1)       1.00      0.87      0.93     19866

    accuracy                        